In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
#Prevent Large File crashing on Jupyter Notebook
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
asinSet= set()
with open('final_goodreads_asin.csv', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        asinSet.add(line[0])

In [4]:

l = []
with open('complete_Books.json', 'r') as f:
    with open('amazon_official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t' + 'reviewText' + '\n')
        for i, line in enumerate(f):
            j = json.loads(line)
            if j['asin'] in asinSet:
                if not 'reviewText' in j: 
                    g.write(str(j['overall']) + '\t' + j['reviewTime'].replace(","," ") + '\t'+ j['asin'] + '\t' + '' + '\n' )
                else :
                    #l.append([j['overall'], j['reviewTime'], j['asin'], j['reviewText']])
                    g.write(str(j['overall']) + '\t' + j['reviewTime'].replace(","," ")  + '\t'+ j['asin'] + '\t' + j['reviewText'].replace("\n"," ") + '\n' )
                
            
    print(j.keys())
    print(i)

dict_keys(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'])
51311620


# ----------------------------------------------------


In [21]:
### l = []
with open('amazon_official.tsv', 'r') as f:
    with open('amazon_textOnly_official.tsv', 'w') as g:
        #g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+','+'reviewText' + '\n')
        for i, line in enumerate(f):
            line = line.strip().split('\t')
            asin = line[2]
            
            if len(line) > 3:
                text = line[3] 
                g.write(asin + '\t' + text + '\n' )
            else:
                g.write(asin + '\t' +''+ '\n' )

# -----------------------------------------------------
## Filter 1 more time (after detecting Goodreads English text )

In [11]:
asinSet= set()
with open('final_goodreads_asin.csv', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        asinSet.add(line[0])

In [12]:
### l = []
count = 0
with open('amazon_cleaned_official.tsv', 'r') as f:
    next(f)
    with open('amazon_cleaned_Official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t'+'reviewText' + '\t' + 'cleaned_text' + '\n')
        for i, line in enumerate(f):
            l = line.strip().split('\t')
            #overall = line[0]
            #reviewTime = line[1]
            asin = l[2]
            if asin in asinSet:
                g.write(line)
                count +=1
        print(count)

5683680


## Calculate metadata

In [43]:
asinDict = {}
textCount = {}
with open('amazon_cleaned_Official.tsv', 'r') as f:
    next(f)
    for i,line in enumerate(f):
        line = line.strip().split('\t')
        overall = float(line[0])
        asin = line[2]
        if not asin in asinDict :
            textCount[asin]=0
            asinDict[asin] =[]
        asinDict[asin].append(overall)
        if len(line) > 3:
            textCount[asin] +=1
        

In [44]:
#Calculate average of each book
average = {}
for asin in asinDict:
    average[asin]=np.round(sum(asinDict[asin])/len(asinDict[asin]),2)

In [45]:
#Number of ratings of each book
rating_count = {}
for asin in asinDict:
    rating_count[asin] = len(asinDict[asin])

In [46]:
#Number of text revirws of each book
text_reviews_count = {}
for asin in asinDict:
    text_reviews_count[asin] = textCount[asin]

In [47]:
result = { asin: [asin,average[asin], rating_count[asin], text_reviews_count[asin] ] for asin in asinDict }

In [48]:
amazon= pd.DataFrame.from_dict(result,orient='index')
amazon.columns = ['asin', 'average', 'rating_count','text_reviews_count']

In [49]:
amazon.reset_index(drop=True)

,asin,average,rating_count,text_reviews_count
0,0001713353,4.83,54,54
1,0001061240,4.87,45,45
2,0001711296,4.44,107,107
3,0002007649,3.37,19,19
4,0001716069,4.61,59,59
...,...,...,...,...
37228,0312943636,4.17,24,24
37229,0312953038,4.00,27,27
37230,0312955154,3.29,14,14
37231,0312956878,3.33,15,15


## Merging with genres dataset

In [50]:
genres = pd.read_csv('genres_amazon.csv')
g = pd.DataFrame(genres)
g.columns

Index(['asin', 'genres', 'rank'], dtype='object')

In [51]:
meta = pd.merge(amazon,g, on='asin', how='inner')

In [52]:
meta.isnull().sum()

asin                    0
average                 0
rating_count            0
text_reviews_count      0
genres                842
rank                    0
dtype: int64

In [53]:
new_meta = meta.copy()

In [54]:
new_meta['genres']=new_meta['genres'].astype(str).apply(lambda st: st[st.find("[")+1:st.find("]")]).str.replace('\'', '').str.replace('\"', '')

In [55]:
new_meta['genres'] = new_meta['genres'].str.replace('Books,','',1).str.replace(' Genre','').str.replace(' Arts','Art')
new_meta['genres'] = new_meta['genres'].str.replace(', & Style','',1).str.replace(', ,','')

In [56]:
new_meta

,asin,average,rating_count,text_reviews_count,genres,rank
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction","1,461,315"
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction","321,557"
2,0001711296,4.44,107,107,Literature & Fiction,"2,884,610"
3,0002007649,3.37,19,19,"Science & Math, Chemistry","9,799,524"
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry","3,841,172"
...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction","2,583,900"
37229,0312953038,4.00,27,27,"Romance, Historical","553,268"
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction","3,412,599"
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime","2,606,128"


In [57]:
new_meta.to_csv('amazon_metadata.csv', index = False)

## Amazon cleaned text only 

In [32]:
### l = []
with open('amazon_cleaned_Official.tsv', 'r') as f:
    with open('amazon_textOnly_official.tsv', 'w') as g:
        #g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+','+'reviewText' + '\n')
        for i, line in enumerate(f):
            #if i == 3:
                #break
            line = line.strip().split('\t')
            asin = line[2]
            if len(line) > 4:
                text = line[4] 
                g.write(asin + '\t' + text + '\n' )

# --------------------------------
## Get more Amazon metadata

In [3]:
asinSet= set()
with open('final_goodreads_asin.csv', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        asinSet.add(line[0])

In [20]:
asinDict = {}
verifiedCount = {}
styleDict = {}

with open('complete_Books.json', 'r') as f:
    for i,line in enumerate(f):
        j = json.loads(line)
        asin = j['asin']
        if asin in asinSet:
            if not asin in verifiedCount:
                verifiedCount[asin]=0
            if j['verified'] == True:
                verifiedCount[asin] +=1
            if not asin in styleDict:
                if 'style' in j: 
                    styleDict[asin] =set()
                    styleDict[asin].add(j['style']['Format:'])
            try:
                if 'style' in j: 
                    styleDict[asin].add(j['style']['Format:'])
            except:
                 styleDict[asin].add('')

In [21]:
verified = pd.DataFrame.from_dict(verifiedCount, orient="index").reset_index()
verified.columns = ['asin', 'verifiedTrue_count']
verified

,asin,verifiedTrue_count
0,0001713353,36
1,0001061240,30
2,0001711296,69
3,0002007649,3
4,0001716069,44
...,...,...
37228,0312943636,9
37229,0312953038,12
37230,0312955154,4
37231,0312956878,9


In [39]:
style = pd.DataFrame(styleDict.keys())
style.columns = ['asin']
style['Format']=styleDict.values()
style['Format']= style['Format'].astype(str).apply(lambda st: st[st.find("{")+1:st.find("}")]).str.replace('\'','')

In [63]:
merge = pd.merge(verified,style, on = 'asin', how = 'left')

In [65]:
df = pd.read_csv('amazon_metadata.csv')

In [66]:
complete = pd.merge(df, merge, on = 'asin', how = 'inner')

In [67]:
complete

,asin,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction","1,461,315",36,"Paperback, Hardcover"
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction","321,557",30,Hardcover
2,0001711296,4.44,107,107,Literature & Fiction,"2,884,610",69,"Library Binding, VHS Tape, Paperback, Hard..."
3,0002007649,3.37,19,19,"Science & Math, Chemistry","9,799,524",3,"Kindle Edition, Paperback, Hardcover"
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry","3,841,172",44,"Kindle Edition, Paperback, Hardcover"
...,...,...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction","2,583,900",9,"Mass Market Paperback, Kindle Edition"
37229,0312953038,4.00,27,27,"Romance, Historical","553,268",12,"Mass Market Paperback, Kindle Edition, Pape..."
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction","3,412,599",4,"Kindle Edition, Paperback"
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime","2,606,128",9,"Mass Market Paperback, Hardcover"


In [68]:
complete.to_csv('complete_amazon_metadata.csv')

# ---------------------------------------------------------

In [29]:
filenames = ['amazon_clean/amazon_cleaned_{}.tsv'.format(k) for k in range(1,501)]

with open('amazon_count_official.tsv','w') as f:
    f.write('overall'+'\t'+'reviewTime'+'\t'+'asin'+'\t'+'count_before' +'\t'+ 'count_after' +'\t'+'reviewText'+'\t'+'cleaned_text'+'\n')
    for file in filenames:
        with open(file,'r') as g:
            next(g)
            for i, line in enumerate(g):
            #if i ==3:
                #break
                text = line.strip().split('\t')
                #print(text)
                if len(text)>4:
                    count_before = len(text[3].split(' '))
                    count_after  = len(text[4].split(' '))
                    f.write(text[0]+'\t'+text[1]+'\t'+text[2]+'\t'+str(count_before)+'\t'+str(count_after)+'\t'+text[3]+'\t'+text[4]+'\n')
                elif (len(text) == 4):
                    count_before = len(text[3].split(' '))
                    count_after = 0
                    f.write(text[0]+'\t'+text[1]+'\t'+text[2]+'\t'+str(count_before)+'\t'+str(count_after)+'\t'+text[3]+'\t'+str(np.nan)+'\n')
                else : 
                    count_before = 0
                    count_after  = 0
                    f.write(text[0]+'\t'+text[1]+'\t'+text[2]+'\t'+str(count_before)+'\t'+str(count_after)+'\t'+str(np.nan)+'\t'+str(np.nan)+'\n')
            
        

In [4]:
asinSet= set()
with open('final_goodreads_asin.csv', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        asinSet.add(line[0])

In [38]:
with open('amazon_count_official.tsv', 'r') as f:
    next(f)
    with open('amazon_cleaned_official.tsv', 'w') as g:
        g.write('overall'+'\t'+'reviewTime'+'\t'+'asin'+'\t'+'count_before' +'\t'+ 'count_after' +'\t'+'reviewText'+'\t'+'cleaned_text'+'\n')
        for i,line in enumerate(f):
            text = line.strip().split('\t')
            asin = text[2]
            if asin in asinSet:
                g.write(line)
            

In [5]:
count_beforeDict={}
count_afterDict = {}
asinDict = {}
with open('amazon_cleaned_official.tsv','r') as g:
    next(g)
    for i, line in enumerate(g):
        text = line.strip().split('\t')
        asin = text[2]
        #print(line)
        count_before = int(text[3])
        count_after  = int(text[4])
        #print(line)
        if not asin in count_beforeDict:
            count_beforeDict[asin] = 0
            count_afterDict[asin] = 0
        
        count_beforeDict[asin] += count_before
        count_afterDict[asin] += count_after 

In [6]:
count_before= pd.DataFrame(count_beforeDict.keys())
count_before.columns = ['asin']
count_before['am_countText_before']=count_beforeDict.values()
count_before

,asin,am_countText_before
0,0001713353,2362
1,0001061240,3085
2,0001711296,5667
3,0002007649,5668
4,0001716069,3081
...,...,...
37228,0312943636,5228
37229,0312953038,3184
37230,0312955154,1456
37231,0312956878,968


In [7]:
count_after= pd.DataFrame(count_afterDict.keys())
count_after.columns = ['asin']
count_after['am_countText_after']=count_afterDict.values()
count_after

,asin,am_countText_after
0,0001713353,1037
1,0001061240,1326
2,0001711296,2574
3,0002007649,2810
4,0001716069,1457
...,...,...
37228,0312943636,2508
37229,0312953038,1420
37230,0312955154,683
37231,0312956878,450


In [8]:
count = pd.merge(count_before,count_after, on ='asin', how = 'inner')
count

,asin,am_countText_before,am_countText_after
0,0001713353,2362,1037
1,0001061240,3085,1326
2,0001711296,5667,2574
3,0002007649,5668,2810
4,0001716069,3081,1457
...,...,...,...
37228,0312943636,5228,2508
37229,0312953038,3184,1420
37230,0312955154,1456,683
37231,0312956878,968,450


In [40]:
df = pd.read_csv('complete_amazon_metadata.csv')
complete = pd.merge(df, count, on = 'asin', how = 'inner')
complete = complete.drop(['Unnamed: 0'],axis=1)
complete

,asin,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format,am_countText_before,am_countText_after
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction","1,461,315",36,"Paperback, Hardcover",2362,1037
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction","321,557",30,Hardcover,3085,1326
2,0001711296,4.44,107,107,Literature & Fiction,"2,884,610",69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
3,0002007649,3.37,19,19,"Science & Math, Chemistry","9,799,524",3,"Kindle Edition, Paperback, Hardcover",5668,2810
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry","3,841,172",44,"Kindle Edition, Paperback, Hardcover",3081,1457
...,...,...,...,...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction","2,583,900",9,"Mass Market Paperback, Kindle Edition",5228,2508
37229,0312953038,4.00,27,27,"Romance, Historical","553,268",12,"Mass Market Paperback, Kindle Edition, Pape...",3184,1420
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction","3,412,599",4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime","2,606,128",9,"Mass Market Paperback, Hardcover",968,450


In [41]:
complete.loc[29265, 'rank'] = 71
complete

,asin,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format,am_countText_before,am_countText_after
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction","1,461,315",36,"Paperback, Hardcover",2362,1037
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction","321,557",30,Hardcover,3085,1326
2,0001711296,4.44,107,107,Literature & Fiction,"2,884,610",69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
3,0002007649,3.37,19,19,"Science & Math, Chemistry","9,799,524",3,"Kindle Edition, Paperback, Hardcover",5668,2810
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry","3,841,172",44,"Kindle Edition, Paperback, Hardcover",3081,1457
...,...,...,...,...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction","2,583,900",9,"Mass Market Paperback, Kindle Edition",5228,2508
37229,0312953038,4.00,27,27,"Romance, Historical","553,268",12,"Mass Market Paperback, Kindle Edition, Pape...",3184,1420
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction","3,412,599",4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime","2,606,128",9,"Mass Market Paperback, Hardcover",968,450


In [50]:
#complete['rank'] = pd.to_numeric(complete.rank,errors ='coerce')
complete['rank'] = complete['rank'].str.replace(',','').astype(int)
complete
#complete.isnull().sum()

,asin,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format,am_countText_before,am_countText_after
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction",1461315,36,"Paperback, Hardcover",2362,1037
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326
2,0001711296,4.44,107,107,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
3,0002007649,3.37,19,19,"Science & Math, Chemistry",9799524,3,"Kindle Edition, Paperback, Hardcover",5668,2810
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry",3841172,44,"Kindle Edition, Paperback, Hardcover",3081,1457
...,...,...,...,...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction",2583900,9,"Mass Market Paperback, Kindle Edition",5228,2508
37229,0312953038,4.00,27,27,"Romance, Historical",553268,12,"Mass Market Paperback, Kindle Edition, Pape...",3184,1420
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450


In [55]:
complete.to_csv('official_amazon_metadata.csv', index = False)